# Errors and Debugging
- Errors == Bugs
- The term "bug" comes from  1945 when scientists were working on the Mark II computer.  An error occurred and the computer stopped.  Upon investigating they found a moth caught in a mechanical switch.  When the moth was removed the computer worked!  They declared, 
> "We have debugged the computer."
- General Debugging Tips:
    - Scale down input to better understand output
        - E.g. import only 5 lines of text instead of an entire book
    - Summarize data.   Easier to see if something has changed.
        - E.g. checking to see if the length of a list has changed is easier then looking at all the values in that list
    - Check data types as these often cause errors
    - Use a wide variety of possible inputs to test code
- We'll cover
    1. Errors, traceback messages, and the call stack
    1. Try except statements, raising errors on purpose, and performing sanity checks with assertions
    1. Logging information that may be useful for debugging 
    1. Viewing the call stack and variables with debuggers
    1. Comparing actual and expected outputs with unit tests
    1. Asking for help on forums like Stack Overflow

---

## Errors and Traceback
- A Python program normally ends when it encounters invalid code.  There are two types of problems:
    1. **`SyntaxError`**--is an problem that occurs because the syntax is wrong.  This could be a missing comma, an extra parenthesis, etc.  The Python error message that pops up tells us the line where Python first noticed the bad syntax, shows the code, and points to a place in the code with a `^` caret.  The line and caret signify where Python first *noticed* that code was bad, but the bad code could occur anywhere on this line before the caret or even on previous lines.  The error message also tells more detail about the syntax error.  `try` and `except` does NOT work on `SyntaxError`.
    2. **`Exception`**--problem that occurs because of something that is NOT a `SyntaxError`.  The problem could be any other type of error.  The Python error message that pops up has the header **"Traceback"**.  `try` and `except` works on exceptions.  There are around 50 exceptions. Here are a few common ones:
   
1. Exception--exception that all others are based upon
1. AttributeError--raised when an attribute reference or assignment fails
1. ImportError--raised when import statement fails to find module definition or when a `from...import` fails to find 1 name that is to be imported
1. ModuleNotFoundError--subset of ImportError raised when import fails to find a module
1. IndexError--raised when a sequence subscript is out of range
1. KeyError--raised when a mapping (dictionary) key is not found in the set of existing keys
1. KeyboardInterrupt--raised when user hits the interrupt key (normally Control-C or Delete)
1. NameError--raised when a local or global name is not found
1. OSError--raised when a function returns a system related error
1. TypeError--raised when the wrong type of data is used as an input of a function or operation
1. ValueError--raised when a built-in function or operation has the correct type, but the input value is out of the acceptable range
1. ZeroDivisionError--raised when we try to divide by zero in division or modulo
1. RuntimeError--raised when an error is detected that doesn't fall into another category

- Errors (except syntax errors) trigger a traceback message

- **Traceback**--error message that is printed.  Includes error message, the line number where Python first *noticed*  (like syntax above) the error, and the sequence of the function calls that led to the error.  Unlike many other languages, in Python, the most recent function calls are at the bottom of the traceback, with the error at the very bottom.  The sequence of function calls in the traceback are related to the call stack.

- **Call Stack**-- mechanism for an interpreter to keep track of its place in a script that calls multiple functions 
    1. When a script calls a function, interpreter adds (*pushes*) it to call stack and starts carrying out function
    1. Any functions called by that function are added (*pushed*) to call stack further up, and run where their calls are reached
    1. When current function is finished, interpreter removes (**pops**) it from the stack and resumes execution where it left off
    1. If the stack takes up more space than it had assigned to it, it results in a **stack overflow** error

- **Push**--add item to collection.  The term push is also used in Git.
- **Pop**--remove most recent item from collection.  Last in, first out.  Pop is also a list method.

![Image of callstack](images/callstack.jpg)

---

**EXAMPLES**

- Below, notice the order of the call stack
    1. In the first 6 lines, functions `a()` and `b()` are defined.  However, nothing is called, and nothing is added to stack.
    1. On line 7, `a()` is called and added to stack.  Python interpreter starts running `a()`.
    1. `b()` is called from within `a()` and added to top off stack.  Python interpreter jumps to `b()`.
    1. Python interpreter runs `b()` until it is finished. `b()` is is removed from stack.
    1. Python interpreter jumps back to `a()` and runs `a()` until it is finished.  `a()` is removed from stack.

In [1]:
def a():
    print("This is a")
    b()
    print("This is a again")
def b():
    print("This is b")
a()

This is a
This is b
This is a again


- Below, notice the order of the call stack from the traceback.  Traceback lets us know,

> "Traceback (most recent call last)"

- My preferred way is to read the last line with the exception first.  Below, it is `Exception: This is raise Exception within function b.`.  Then jump to the top and read chronologically. Below, we can see:
    1. `in <module>` level (global scope), `a()` is called
    1. `in a()` level (enclosing scope), `b()` is called
    1. `in b()` level (local scope), `raise Execption()` occurs
- Each of the three chunks is called a **frame summary** and show information about a frame object.  The **frame object** is a temporary object created when a function is called and destroyed when the function ends.  They hold local variables and line information that helps the Python interpreter keep track of the stack.

In [3]:
def a():
    print("This is a")
    b()
    print("This is a again")
def b():
    print("This is b")
    raise Exception("This is raise Exception within function b.")
a()

This is a
This is b


Exception: This is raise Exception within function b.

---

## Try and Except
- **`try` `except`**--prevent an exception from stopping the program ("catch an error").  Used when the error is relatively unimportant or can be fixed.
- **`raise`**--trigger an exception on purpose if condition evaluates as *True*, stopping the program.  Avoid a logical error.  Used with`if` statements in production code.  "It is better to fail fast."
- **`assert`**--trigger an exception on purpose if condition evaluates as *False*, stopping the program.  Avoid a logical error. Used with unit testing in code development. Can be read as, "I assert that condition holds True, and if not, there is a logical error so stop the program." 
- **Logical error**--unintended result from valid Python code.  Because the code is valid no exception is raised, but results are still illogical.
    - E.g. Program asks for pet age with `input()` function.  The programmer wants age in years.  The user inputs pet age in months.  The `input()` function worked correctly, but the input will likely cause illogical results.
    - E.g. Program implements a formula to find the average but messes up the parentheses.  Instead of `(x+y) / 2` they write `x + y/2`.  This is valid code, but will cause illogical results.


**Try except basic grammar**
```python
try:
    <RUN THIS CODE>
except:
    <RUN THIS CODE>
```

**Try except complex grammar**
```python
try:
    <RUN THIS CODE>
except <EXCEPTION_NAME> as <ALIAS>
    <RUN THIS CODE>
else:
    <RUN THIS CODE>
finally:
    <RUN THIS CODE>
```

**Raise grammar**
```python
if <CONDITION>:
    raise Exception("Custom error message.")
 ```

**Assert grammar**
```python
assert <CONDITION>, "Custom error message."
```

Code | Use
--- | ---
`try` | Always runs.  Like `if`.  Used to try out code that may create an error.  Give the code the old college try.  If it works, great!
`except` | If try catches an error, then except runs.  Allows a program to keep running instead of stopping.  Except may print a message for the user, may log the exception into a file, may provide a value for future lines of code to use because user input was bad, etc.  Multiple `except` statements can be written, but only one is run (like `elif`).  **Bare exception** is when `except` is used with no other, more specific, Python exception.  It is better programming practice to write one of the ~50 exceptions in the `except` statement to tailor an action to a specific problem.
`else` | Runs if no exceptions were raised.  Kinda like `else` used in `if` statements, but both `try` and `else` will run.  In `if` statement only  `else` runs.
`finally` | Always runs, whether or not there is an exception
`as` | When an exception occurs, Python creates an exception object.  We can examine the exception object by assigning it a variable name (an alias) using `as`.  Then we can work with that variable in subsequent statements.
`raise Exception()` | Trigger a bare exception with custom message.  Custom message is optional.  Used with if statement so exception triggered if condition is *True*.
`assert` | Trigger `AssertionError` exception and custom message if condition evaluates to *False*.  Custom message is optional.
`python -O <FILENAME>.py` | Run script while skipping assertions

---

**EXAMPLES**

**`try except`**

- try except does NOT help with syntax errors

In [4]:
print"We are missing the first parenthesis.")

SyntaxError: invalid syntax (Temp/ipykernel_15376/3832458768.py, line 1)

In [5]:
try:
    print"We are missing the first parenthesis.")
except:
    print("If you are reading this then try failed, but except worked.")

SyntaxError: invalid syntax (Temp/ipykernel_15376/693241023.py, line 2)

- try except DOES help with all other errors/exceptions

In [6]:
# Exception example with a missing t in print
prin("Hello world")

NameError: name 'prin' is not defined

In [7]:
# Exception with try and except.  Bare except.
try:
    prin("If you are reading this then try worked.")
except:
    print("If you are reading this then try failed, but except worked.")

If you are reading this then try failed, but except worked.


In [8]:
# Exception with try and except.  Except statement uses specific Python exception.
try:
    prin("If you are reading this then try worked.")
except NameError:
    print("If you are reading this then try failed, but except worked.")

If you are reading this then try failed, but except worked.


In [9]:
# Exception with try and except.  Except statement uses the WRONG specific Python exception so still fails.
try:
    prin("If you are reading this then try worked.")
except TypeError:
    print("If you are reading this then try failed, but except worked.")

NameError: name 'prin' is not defined

In [10]:
# Exception with try and except
# Except statement uses the WRONG specific Python exception, but another except works.
# Bare except used at end as catch all. Not needed in this example.
try:
    prin("If you are reading this then try worked.")
except TypeError:
    print("If you are reading this then try failed, but one of the excepts worked.")
except NameError:
    print("If you are reading this then try failed, but one of the excepts worked.")
except:
    print("If you are reading this then try failed, but one of the excepts worked.")

If you are reading this then try failed, but one of the excepts worked.


**`else`**

In [11]:
try:
    print("If you are reading this then try worked.")
except TypeError:
    print("If you are reading this then try failed, but except worked.")
else:
    print("This is else.  If you are reading this then try worked.")

If you are reading this then try worked.
This is else.  If you are reading this then try worked.


**`finally`**

In [12]:
# finally with no exceptions
try:
    print("If you are reading this then try worked.")
except:
    print("If you are reading this then try failed, but except worked.")
finally:
    print("This is finally.  Regardless of what happended, finally runs.")

If you are reading this then try worked.
This is finally.  Regardless of what happended, finally runs.


**`as`**

In [13]:
try:
    prin("If you are reading this then try worked.")
except NameError as T_time:
    print(type(T_time))
    print(T_time)
    print("If you are reading this then try failed, but except worked.")

<class 'NameError'>
name 'prin' is not defined
If you are reading this then try failed, but except worked.


**`raise Exception()`**

In [14]:
raise Exception()

Exception: 

In [15]:
greeting = "Hello whirled"
if greeting != "Hello world":
    raise Exception('This is an Exception.  Wrong celestial salutation.  Please address the "world."  Thank you.')

Exception: This is an Exception.  Wrong celestial salutation.  Please address the "world."  Thank you.

In [16]:
# Raise is often used within functions
def age_in_50_years(current_age):
        if current_age <= 0:
            raise Exception("Age can not be negative...yet")
        elif current_age >= 150:
            raise Exception("No human lives this long...yet.")
        new_age = current_age + 50
        return new_age

# Try except is often used when functions have a raise
try:
    age_in_50_years(-1)
except Exception as err:
    print(f'An exception has occured. {err}')

An exception has occured. Age can not be negative...yet


**`assert`**

In [17]:
assert False

AssertionError: 

In [18]:
greeting = "Hello whirled"
assert greeting == "Hello world", 'I assert this is True, if not stop the program.' 

AssertionError: I assert this is True, if not stop the program.

In [19]:
sorted_numbers = sorted([3, 4, 1, 5, 2])
assert sorted_numbers[0] < sorted_numbers[4] , 'I assert this is True, if not stop the program.' 

---

## Logging
- The logging module in Python is very complicated (also sometimes uses dromedary camelCase) as it is copied from old Java code
- **Logging**--log data and descriptive message at different lines in a script.  Similar to using a print statement to display a variable's value at different lines in a script.  Using a logging module provides many more options and is recommended.
- **Log Record**--each time data + message + level is recorded it forms a log record
- **Logging levels**--categorize log messages by importance.  There are 5 logging levels.  Messages can be logged at each level using a different logging function.

![](images/logging.jpg)

- Logging may be combined with try except statements so that when an exception is caught, a log record is created.  Likely would be level error or critical
- **Logger**--creates log records.  Specify which levels to log, while rest are ignored.
- **Handler**--controls whether log records are printed, saved to a file, or sent across a network.  Like logger, handler can also specify which levels to log.
- **Filters**--additional options that control which log records are logged and which are ignored
- **Formatter**--controls the text format of the log record.  We do this by string formatting that is similar (but different) to Python's now deprecated conversion specifier `%s`.   E.g. `%(asctime)s -  %(levelname)s -  %(message)s'`.  Can change date-time format if needed.

![](images/logging_attributes.jpg)

- The logger, handler, filter, and formatter all have options that can be changed.  These options are stored in an underlying Python dictionary.  This dictionary can be:
    1. Created directly
    1. Created through the use of an external JSON or YAML file
    1. Created with Python functions (what we'll do)

Code | Use
--- | ---
`logging` | Module
`logging.getLogger(__name__)` | Create logger object.   By convention, argument is`__name__`.
`__name__` |  Variable assigned to string value that changes depending on whether the code line containing `__name__` is run as a script (what we do normally) or is imported as a module.  When the containing code is run as a script it equals `'_main__'`.  When containing code is imported as module `__name__` will be a string of the imported module's name.
`.setLevel(logging.<LEVEL>)` | Logger object method.  Set level.  Level in all caps.  Logs specified level and all levels above (more important), while ignoring the rest.  E.g. `DEBUG` records all levels.  `ERROR` records error and critical. 
`logging.StreamHandler()` | Create stream handler object.  Prints log records.
`logging.FileHandler('<FILENAME.EXT>', '<MODE>')` | Create file handler object.  Saves log records to file.
`.setLevel(logging.<LEVEL>)` | Handler object method.  Same grammar as logging object.
`logging.Formatter(<FORMAT>)` | Create formatter object
`'%(asctime)s - %(name)s - %(levelname)s - %(message)s'` | Example format
`.setFormatter(<FORMATTER_OBJECT_NAME>)` | Handler object method.  Add formatter to handler.
`.addHandler(<HANDLER_OBJECT_NAME>)` | Logger object method.  Add handler to logger object.
`.debug('<MESSAGE>')` | Logger object method.  Debug level log record object created.
`.info('<MESSAGE>')` | Logger object method.  Info level log record object created.
`.warning('<MESSAGE>')` | Logger object method.  Warning level log record object created.
`.error('<MESSAGE>')` | Logger object method.  Error level log record object created.
`.critical('<MESSAGE>')` | Logger object method.  Critical level log record object created.

---

**EXAMPLES**

In [20]:
import logging

**Configuration**

In [21]:
# Create logger object
logger_object = logging.getLogger(__name__)
logger_object.setLevel(logging.DEBUG)

# Create handler object
stream_handler_object = logging.StreamHandler()  # Prints to console/terminal
file_handler_object = logging.FileHandler('./output/log_records.txt', 'a')  # Saves to file
stream_handler_object.setLevel(logging.DEBUG)
file_handler_object.setLevel(logging.DEBUG)

# Create formatter object
formatter_object = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Add formatter to handler object
stream_handler_object.setFormatter(formatter_object)
file_handler_object.setFormatter(formatter_object)

# Add handler object to logger object
logger_object.addHandler(stream_handler_object)
logger_object.addHandler(file_handler_object)

**Log Records**

In [22]:
logger_object.debug('debug message')
logger_object.info('info message')
logger_object.warning('warn message')
logger_object.error('error message')
logger_object.critical('critical message')

2022-01-01 21:34:03,116 - __main__ - DEBUG - debug message
2022-01-01 21:34:03,117 - __main__ - INFO - info message
2022-01-01 21:34:03,118 - __main__ - WARNING - warn message
2022-01-01 21:34:03,119 - __main__ - ERROR - error message
2022-01-01 21:34:03,119 - __main__ - CRITICAL - critical message


**Capturing Traceback Message**

In [23]:
try:
    2/0
except:
    logger_object.error("Exception occured.  Traceback message is:", exc_info=True)

2022-01-01 21:34:03,659 - __main__ - ERROR - Exception occured.  Traceback message is:
Traceback (most recent call last):
  File "C:\Users\CHRISA~1\AppData\Local\Temp/ipykernel_15376/1648813604.py", line 2, in <module>
    2/0
ZeroDivisionError: division by zero


In [24]:
# The following does the same as the above
# The above can use any level (even though we use error)
# The following always creates log records of level error, but has cleaner looking code
try:
    2/0
except:
    logger_object.exception("Exception occured")

2022-01-01 21:34:03,934 - __main__ - ERROR - Exception occured
Traceback (most recent call last):
  File "C:\Users\CHRISA~1\AppData\Local\Temp/ipykernel_15376/2542281761.py", line 5, in <module>
    2/0
ZeroDivisionError: division by zero


---

## Debugging
- **Debugging**--the process of fixing mistakes in code
- **Debugger**--program that helps users debug code.  The basic idea is that we execute our code line by line.  At each line we can:
    1. See the callstack
    1. See currently existing variable values and their data type in the different scopes (local, global, and built-in)
- Common terminology:
    - **Step over**--execute 1 line then pause.  If we use step over on a line with a function call, we execute the entire function as if it were a black box and then pause at the line underneath that function
    - **Step into**--execute 1 line then pause.  If we use step into on a line with a function call, we go into that function and pause on the first line.  The twist is that  this only steps into functions defined in our code.  You can't step into functions from other imported modules. 
    - **Step Out**--executes all the code in the current function, steps out of current function, and pauses at next line.  Allows us to quickly step out of current function we've stepped into.
    - **Breakpoints**--location (point) in a script we want to stop at (break into debugger).  Stepping line by line through code could take a long time.  Setting breakpoints at points of interest allow us to run many lines of code and then stop at the breakpoint.  We could then inspect the callstack or variables.
    - **Continue**--execute lines of code until we reach next breakpoint, then pause
    - **Post Mortem Debugging**--debugging after a program (script) has already crashed
- We'll first cover the `repr()` function and how it differs form the `str()` function.  This is relevant because when debuggers display variable values, they use `repr()` under the hood.
- Then we'll look at a few Python debuggers like pdb, Jupyter lab, and VS Code.

Code | Use
 --- | ---
 `str(<OBJECT>)` | Based upon `__str__` and returns very readable "unofficial" string version of specified object
 `repr(<OBJECT>)` | Based upon `__repr__` and returns unambiguous "official" string version of specified object.  Good for debugging purposes.

---

**EXAMPLES**

In [25]:
print(str(123))
print(repr(123))  # No difference

123
123


In [26]:
print(str("Hello world"))  # Removes quotes for readability
print(repr("Hello world"))  # Notice the quotes

Hello world
'Hello world'


In [27]:
import datetime

today = datetime.datetime.now()
print(str(today))  # Easier to read
print(repr(today))  # More info on data type

2022-01-01 21:34:06.401776
datetime.datetime(2022, 1, 1, 21, 34, 6, 401776)


In [28]:
try:
    2/0
except ZeroDivisionError as err:
    print(str(err))  # Less detailed
try:
    2/0
except ZeroDivisionError as err:
    print(repr(err))   # More detailed

division by zero
ZeroDivisionError('division by zero')


---

**Python Debugger**

- **pdb**--Python debugger.  Debugger in Python Standard Library. Everything can be done through the command line, so it's great if we do not have access to an IDE.  This may occur if we are debugging on a server. IDEs have more advanced GUI debuggers that should be used when available.

![](images/pdb_commands.jpg)
 
- Along with the commands above, a few pieces of grammar to know are:
 
Code | Use
--- | ---
`pdb` | Module
`python -m pdb <SCRIPT_NAME>.py` | Start debugging from terminal
`(Pdb)` | Prompt, signifying debugger is read to to accept next command.  Similar to `>>>` in REPL.
 `>` | Starts the first line and tells us which source file path we're in.  After the filename, there is a current line number in parentheses.  Next is the name of the function.  If we are not inside a function, then we are at the module level and see `<module>()`.
`->` | Signifies the line that we are paused right before.  We have not run this line yet.
`--Call--` |  When we step into a function that is called this text is shown
`--Return--` |  Both `n` and `s` stop at end of a function and print return value at the end of the next line after ->

---

**EXAMPLES**

- The easiest way to learn how to use the debugger is to start debugging from the terminal by calling `python -m pdb <SCRIPT_NAME>.py` and playing with the commands
- Try the `pdb_example_1.py` file in the input folder
    - Note that `p` uses `__repr__` under the hood
    - As a reminder, if accidently step into a module `u` can be used to move up one level
- Try the `pdb_example_2.py` file in the input folder
   - Note the scope of variabes.  The global variable can always be printed after the assignment statement is executed.  The local variable can only be printed after the assignment statement has been executed while that function is running.  See the *Namespace and Scope* section for the logic behind this.
- Try the `pdb_example_3.py` file in the input folder
    - Note the exceptions and how they are displayed when we go through them (strangely).  This is on top of return values which are always displayed multiple times at different levels (strangely).  Not intuitive.

---

**Jupyter Lab Debugger**
- Jupyter Lab has its own debugger starting in Jupyter Lab 3.0
- The ipykernel has support for this debugger
    1. Enable debugger by clicking on an icon in the top right
    1. Add breakpoint by clicking to the left of the line number
    1. Run the cell normally
    1. Inspect the panels to the right
        - Callstack panel has GUI equivalent to `c`, `n`, `s`, `q`, and a couple others.
        - Variables panel can display global variables (lots if we have tons of cells) and local variables
        - Breakpoints panel shows line numbers of breakpoints.  Not helpful as these are seen to left of code cell anyway
        - Source panel shows actual code and highlights the lines as we step through them  

---

**EXAMPLES**

In [29]:
global_variable_text = "Hello world"
def a():
    print("This is a")
    local_variable_text = "Hello moon"
    b()
    print("This is a again")
    return "This string is the function a return value"
def b():
    print("This is b")
    return "This string is the function b return value"
a()

This is a
This is b
This is a again


'This string is the function a return value'

---

**VS Code Debugger**
- VS Code has a good GUI debugger
- For a simple debug
    1. Open script
    1. Create a breakpoint on the first line by clicking to the left of the line number
    1. Run the script in debug mode. This can be done in either of 3 ways.
    1. ![](images/dbg_vscode.jpg)
    1. Control the steps with the floating commands bar, which has GUI equivalents to `c`, `n`, `s`, `q`, and a couple others.
    1. Inspect the variables and callstack panel to the left
- Try out the `pdb_example_2.py` mentioned above
- Note that there are many more advanced options and features in VS Code

---

## Unit Testing
- **Unit testing**--processes of running code chunks (**units**) with a wide range of inputs and comparing observed outputs to expected outputs (**test assertion**).  Units are often functions or methods.  Similar to making multiple assert statements like we've seen above, but in a more automated way.  
- **Integration testing**--tests how multiple code chunks work together.  If integration test fails, we do not know which code chunk failed, only that one did.  Unit tests can then be done.
- The meaning of many other terms will vary depending on the source.  These include: test, test step, test case, test script, test suite, test plan,  etc.  The general concepts are more important than the terminology one chooses.
- **Test runner/test framework**--special application that automates the running of tests.  Also provides diagnostics and debugging tools when tests fail.  The two most popular ones are `unittest` and `pytest`.
1. `unittest`:
    1. Included in Python Standard Library
    1. Tests defined in class methods
    1. Must use a series of special assertion methods instead of the built-in `assert` statement
    1. More verbose, but easier to read for some
- `pytest`:
    1. Must be installed
    1. Tests defined in functions.  Optionally, define related tests as methods in a class.
    1. Uses built-in assert statement
    1. Less verbose
    1. Many advanced features
- We'll choose `pytest` because of the simpler grammar.  A potential `pytest` workflow includes:
    1. Define test functions in separate script. Test function names starts with `test_`.
    1. Save script.  Script name must either start with `test_` or end with `_test.py`.
    1. In a terminal, enter `pytest <PATH>/test_<REST_OF_NAME>.py`.  Tests are then run and results are returned in the terminal. Alternatively, navigate to the script current working directory or parent folder and enter `pytest`.  Pytest will look in the current directory and walk subdirectories until it finds scripts with the correct naming scheme.  It then runs the tests in those scripts.
        - We never need to activate the interactive interpreter.  We never need to `import pytest`.

---

**EXAMPLES**

**Manual Unit Tests**

In [30]:
my_input = [1, 2, 3]  # List.
assert sum(my_input) == 6, "Error.  Must sum to  6"  # Raises AssertionError if condition evaluates as False

In [31]:
my_input = (1, 2, 3)  # Tuple.
assert sum(my_input) == 6  # Custom AssertionError message is optional

In [32]:
my_input = {1, 2, 3}  # Set.
assert sum(my_input) == 6, "Error.  Must sum to  6"

**Pytest**

- Below we define test functions.  These definitions do not do anything on their own.  They would be included in a separate script with a name like `test_example.py` and run with the command `pytest <PATH>/test_example.py`

In [33]:
def test_list_input():
    my_input = [1, 2, 3]  # List.
    assert sum(my_input) == 6, "Error.  Must sum to  6"

def test_tuple_input():
    my_input = (1, 2, 3)  # Tuple.
    assert sum(my_input) == 6 # Custom AssertionError message is optional

def test_set_input():
    my_input = {1, 2, 3}  # Set.
    assert sum(my_input) == 6, "Error.  Must sum to  6"

---

## Asking for Help
- When all else fails we can always search Google and Stack Overflow.  If we have a question in Python, it has likely already been asked and answered on Stack Overflow.  If not, we can ask.  Here are some tips:
    1. Summarize question in headline
    1. State question in the form of a question
    1. If not obvious from question, explain what we want code to do
    1. Include full exception message we are getting
    1. Share **minimal, complete, and reproducible (MCR)** example code that returns the same exception we are getting.  The MCR example can included within the forum, or we can link to code saved on [Paste Bin](https://pasetbin.com) or as [GitHub Gists](https://gist.github.com/discover).
    1. Explain what we've already tried
    1. Describe our setup including OS name + version, Python interpreter version, third-party modules used by our script + their versions, and if we are using a virtual environment

---